In [1]:
import jsonlines
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
matches = []
with jsonlines.open('matches.jl') as reader:
    for obj in reader:
        matches.append(obj)

In [3]:
len(matches)

5049

In [4]:
head = ["team1_name", "team2_name", "best_of", "match_type", "date", "timestamp", "prize", "teams", "location"]
head += [f"map{i}" for i in range(1,6)]
head += [f"team{team}_player{p}_{attr}" for team in range(1,3) for p in range(1,6) for attr in ["name", "rating", "DPR", "KAST", "impact", "ADR", "KPR"]]
head += ["team1_points", "team2_points", "team1_wins", "team2_wins"]

In [5]:
records = []
for match in matches:
    record = []
    record.append(match["team1"]["name"])
    record.append(match["team2"]["name"])
    record.append(match["best_of"])
    record.append(match["match_type"])
    record.append(match["date"])
    record.append(match["unix_time"])
    record.append(match["event"]["prize"])
    record.append(match["event"]["teams"])
    record.append(match["event"]["location"])
    maps = match["maps"]
    for i in range(5):
        if i < len(maps):
            record.append(maps[i])
        else:
            record.append(None)
    for player in match["lineup1"]:
        for attr in player.keys():
            record.append(player[attr])
    for player in match["lineup2"]:
        for attr in player.keys():
            record.append(player[attr])
    record.append(match["team1"]["points"])
    record.append(match["team2"]["points"])
    record.append(match["team1"]["points"] > match["team2"]["points"])
    record.append(match["team2"]["points"] > match["team1"]["points"])
    records.append(record)
            
df = pd.DataFrame(records, columns = head)

In [6]:
df.sample(5)

,team1_name,team2_name,best_of,match_type,date,timestamp,prize,teams,location,map1,map2,map3,map4,map5,team1_player1_name,team1_player1_rating,team1_player1_DPR,team1_player1_KAST,team1_player1_impact,team1_player1_ADR,team1_player1_KPR,team1_player2_name,team1_player2_rating,team1_player2_DPR,team1_player2_KAST,team1_player2_impact,team1_player2_ADR,team1_player2_KPR,team1_player3_name,team1_player3_rating,team1_player3_DPR,team1_player3_KAST,team1_player3_impact,team1_player3_ADR,team1_player3_KPR,team1_player4_name,team1_player4_rating,team1_player4_DPR,team1_player4_KAST,team1_player4_impact,team1_player4_ADR,team1_player4_KPR,team1_player5_name,team1_player5_rating,team1_player5_DPR,team1_player5_KAST,team1_player5_impact,team1_player5_ADR,team1_player5_KPR,team2_player1_name,team2_player1_rating,team2_player1_DPR,team2_player1_KAST,team2_player1_impact,team2_player1_ADR,team2_player1_KPR,team2_player2_name,team2_player2_rating,team2_player2_DPR,team2_player2_KAST,team2_player2_impact,team2_player2_ADR,team2_player2_KPR,team2_player3_name,team2_player3_rating,team2_player3_DPR,team2_player3_KAST,team2_player3_impact,team2_player3_ADR,team2_player3_KPR,team2_player4_name,team2_player4_rating,team2_player4_DPR,team2_player4_KAST,team2_player4_impact,team2_player4_ADR,team2_player4_KPR,team2_player5_name,team2_player5_rating,team2_player5_DPR,team2_player5_KAST,team2_player5_impact,team2_player5_ADR,team2_player5_KPR,team1_points,team2_points,team1_wins,team2_wins
926,Sinners,Anonymo,Best of 3 (Online),* Swiss round 3 (teams with a 1-1 record),2021-03-31,1617185400000,"$68,000",20,Europe (Online),Nuke,Inferno,Vertigo,None,None,oskar,1.13,0.64,0.703,1.23,78.5,0.77,ZEDKO,0.99,0.67,0.708,1.04,74.4,0.67,beastik,1.04,0.67,0.704,1.04,72.9,0.66,SHOCK,1.04,0.65,0.707,0.96,72.0,0.67,NEOFRAG,1.20,0.67,0.722,1.29,87.5,0.77,Snax,1.04,0.66,0.687,1.07,74.5,0.70,innocent,1.01,0.68,0.700,1.04,77.5,0.70,mynio,0.93,0.71,0.673,1.05,77.5,0.65,KEi,1.12,0.70,0.694,1.21,82.8,0.74,Kylar,1.07,0.67,0.706,1.05,75.3,0.70,2,0,True,False
4012,W7M,DETONA,Best of 3,* Upper bracket quarter-final,2020-06-11,1591912800000,"$5,000",8+,South America (Online),Inferno,Nuke,Dust2,None,None,real,1.09,0.65,0.727,1.05,75.4,0.68,raafa,1.07,0.63,0.691,1.07,71.9,0.69,pancc,1.12,0.67,0.702,1.18,80.9,0.74,skullz,1.03,0.68,0.705,0.97,73.2,0.66,Tuurtle,1.06,0.70,0.701,1.09,77.4,0.69,vsm,1.22,0.66,0.725,1.31,85.7,0.80,Lucaozy,1.03,0.65,0.699,0.98,71.5,0.67,nak,0.92,0.66,0.699,0.89,70.9,0.61,piriaz1n,1.04,0.69,0.683,1.10,74.2,0.67,zevy,1.14,0.63,0.708,1.17,75.6,0.77,0,2,False,True
2558,Sharks,BOOM,Best of 3,* Group stage,2020-10-04,1601850600000,"$15,000",6,South America (Online),Vertigo,Mirage,Train,None,None,exit,1.14,0.62,0.738,1.04,79.5,0.73,leo_drk,1.08,0.69,0.700,1.15,78.2,0.70,jnt,1.04,0.63,0.706,1.01,69.6,0.65,Luken,1.16,0.64,0.726,1.16,81.4,0.75,supLex,0.98,0.67,0.691,0.94,67.7,0.63,boltz,1.07,0.64,0.722,1.09,81.9,0.71,felps,1.08,0.70,0.706,1.25,84.6,0.76,yel,1.02,0.63,0.705,1.07,71.9,0.68,chelo,1.06,0.66,0.718,1.13,77.8,0.71,shz,1.11,0.64,0.721,1.06,77.8,0.73,0,2,False,True
1370,Nemiga,SAW,Best of 3 (Online),* Swiss round 4 (teams with a 2-1 record). Win...,2021-03-04,1614870600000,"$68,000",20,Europe (Online),Inferno,Vertigo,Dust2,None,None,mds,1.05,0.66,0.703,1.01,73.1,0.68,Jyo,1.08,0.65,0.716,1.05,74.9,0.69,boX,1.03,0.69,0.691,1.10,75.1,0.65,lollipop21k,1.14,0.67,0.724,1.13,82.4,0.76,iDISBALANCE,1.12,0.63,0.711,1.13,73.8,0.71,arki,0.93,0.67,0.692,0.96,71.0,0.63,MUTiRiS,0.99,0.69,0.704,1.10,78.8,0.69,rmn,0.98,0.69,0.701,1.04,74.1,0.68,JUST,1.03,0.65,0.721,1.02,75.5,0.69,stadodo,1.10,0.65,0.707,1.15,75.0,0.69,2,1,True,False
1571,Vitality,Complexity,Best of 3 (Online),* Group B upper bracket semi-final,2021-02-08,1612798200000,"$150,000",12,Europe (Online),Mirage,Overpass,Nuke,None,None,shox,1.08,0.66,0.697,1.10,76.5,0.74,RpK,0.97,0.68,0.704,0.96,72.1,0.67,apEX,1.02,0.70,0.689,1.10,79.3,0.72,ZywOo,1.34,0.62,0.749,1.47,89.2,0.85,misutaaa,1.00,0.67,0.708,0.94,68.1,0.6

In [7]:
df.to_csv("matches.csv")

In [8]:
df.describe(include = 'object')

,team1_name,team2_name,best_of,match_type,date,prize,teams,location,map1,map2,map3,map4,map5,team1_player1_name,team1_player2_name,team1_player3_name,team1_player4_name,team1_player5_name,team2_player1_name,team2_player2_name,team2_player3_name,team2_player4_name,team2_player5_name
count,5049,5049,5049,5049,5049,5049,5044,5044,5049,4443,4429,98,98,5049,5049,5049,5049,5049,5049,5049,5049,5049,5049
unique,495,603,13,480,399,331,25,51,10,11,12,8,8,687,784,824,840,898,795,911,933,951,1001
top,SKADE,forZe,Best of 3,* Quarter-final,2021-04-15,"$50,000",16,Europe (Online),Inferno,Inferno,Inferno,Mirage,Inferno,niki1,tiziaN,xsepower,Rainwaker,k1to,Spiidi,Jerry,Lucky,JUST,magixx
freq,82,70,2627,437,39,580,1438,3129,909,787,840,21,17,68,64,49,62,64,64,47,58,57,49


In [9]:
# I will drop team1_name, team2_name, and player names because names are highly different
df = df.drop(columns=["team1_name", "team2_name"] + [f"team{team}_player{p}_name" for team in range(1,3) for p in range(1,6)])
df.describe(include = 'object')

,best_of,match_type,date,prize,teams,location,map1,map2,map3,map4,map5
count,5049,5049,5049,5049,5044,5044,5049,4443,4429,98,98
unique,13,480,399,331,25,51,10,11,12,8,8
top,Best of 3,* Quarter-final,2021-04-15,"$50,000",16,Europe (Online),Inferno,Inferno,Inferno,Mirage,Inferno
freq,2627,437,39,580,1438,3129,909,787,840,21,17


In [10]:
# drop match type, because of the scrapped format is too diverse
df = df.drop(columns=["match_type"])
df.describe(include = 'object')

,best_of,date,prize,teams,location,map1,map2,map3,map4,map5
count,5049,5049,5049,5044,5044,5049,4443,4429,98,98
unique,13,399,331,25,51,10,11,12,8,8
top,Best of 3,2021-04-15,"$50,000",16,Europe (Online),Inferno,Inferno,Inferno,Mirage,Inferno
freq,2627,39,580,1438,3129,909,787,840,21,17


In [11]:
# map prize to money prize and qualification
pattern = re.compile("^[$][0-9,]*$")
df['prize_money'] = df['prize'].map(lambda x: int(x.replace("$", "").replace(",", "")) if pattern.search(x) else 0)
df['prize_qualification'] = df['prize'].map(lambda x: int(not pattern.search(x)))
df.head()

,best_of,date,timestamp,prize,teams,location,map1,map2,map3,map4,map5,team1_player1_rating,team1_player1_DPR,team1_player1_KAST,team1_player1_impact,team1_player1_ADR,team1_player1_KPR,team1_player2_rating,team1_player2_DPR,team1_player2_KAST,team1_player2_impact,team1_player2_ADR,team1_player2_KPR,team1_player3_rating,team1_player3_DPR,team1_player3_KAST,team1_player3_impact,team1_player3_ADR,team1_player3_KPR,team1_player4_rating,team1_player4_DPR,team1_player4_KAST,team1_player4_impact,team1_player4_ADR,team1_player4_KPR,team1_player5_rating,team1_player5_DPR,team1_player5_KAST,team1_player5_impact,team1_player5_ADR,team1_player5_KPR,team2_player1_rating,team2_player1_DPR,team2_player1_KAST,team2_player1_impact,team2_player1_ADR,team2_player1_KPR,team2_player2_rating,team2_player2_DPR,team2_player2_KAST,team2_player2_impact,team2_player2_ADR,team2_player2_KPR,team2_player3_rating,team2_player3_DPR,team2_player3_KAST,team2_player3_impact,team2_player3_ADR,team2_player3_KPR,team2_player4_rating,team2_player4_DPR,team2_player4_KAST,team2_player4_impact,team2_player4_ADR,team2_player4_KPR,team2_player5_rating,team2_player5_DPR,team2_player5_KAST,team2_player5_impact,team2_player5_ADR,team2_player5_KPR,team1_points,team2_points,team1_wins,team2_wins,prize_money,prize_qualification
0,Best of 3 (Online),2021-06-02,1622628000000,"$68,000",20,Europe (Online),Mirage,Inferno,Overpass,None,None,1.09,0.65,0.725,1.11,80.5,0.74,0.97,0.66,0.713,1.05,75.5,0.65,1.03,0.66,0.714,1.05,75.6,0.70,1.16,0.63,0.727,1.18,77.7,0.75,1.14,0.67,0.715,1.18,80.8,0.74,1.01,0.65,0.716,1.04,75.1,0.68,1.03,0.67,0.730,1.12,77.2,0.70,1.12,0.62,0.737,1.08,77.9,0.75,1.10,0.64,0.714,1.08,75.3,0.70,1.11,0.61,0.715,1.08,72.6,0.71,2,0,True,False,68000,0
1,Best of 3 (Online),2021-05-28,1622225700000,"$9,630",4+,Europe (Online),Vertigo,Dust2,Overpass,None,None,1.03,0.68,0.706,0.98,74.1,0.67,0.96,0.69,0.686,0.89,69.3,0.63,0.94,0.72,0.657,0.94,70.6,0.63,1.03,0.67,0.677,1.04,71.3,0.67,0.98,0.73,0.677,1.00,72.7,0.64,1.02,0.69,0.700,1.07,77.5,0.71,1.05,0.68,0.708,1.02,76.1,0.69,0.97,0.68,0.696,0.85,71.8,0.63,1.01,0.68,0.698,0.99,72.7,0.65,0.95,0.73,0.652,1.02,70.3,0.64,2,0,True,False,9630,0
2,Best of 3 (Online),2021-05-28,1622225100000,8 Spots in Elisa Invitational Summer,16,Europe (Online),Nuke,Vertigo,Inferno,None,None,1.00,0.68,0.702,1.04,73.6,0.68,1.05,0.67,0.710,1.00,74.4,0.67,0.97,0.69,0.693,0.96,71.2,0.59,1.00,0.69,0.696,0.99,71.9,0.63,1.04,0.62,0.708,1.02,66.4,0.65,1.00,0.67,0.704,1.01,76.3,0.68,0.92,0.67,0.687,0.90,69.5,0.62,1.00,0.68,0.695,0.97,71.8,0.63,1.05,0.68,0.677,1.13,71.3,0.70,0.90,0.72,0.655,0.90,67.9,0.58,2,1,True,False,0,1
3,Best of 3 (Online),2021-05-28,1622228400000,"$50,000",16,Europe (Online),Mirage,Inferno,Dust2,None,None,1.01,0.68,0.706,1.09,75.8,0.69,1.15,0.67,0.711,1.29,85.4,0.80,1.13,0.67,0.723,1.25,85.6,0.78,1.03,0.63,0.717,1.03,75.6,0.68,1.06,0.64,0.735,1.04,77.6,0.71,1.16,0.62,0.713,1.28,80.5,0.78,1.03,0.67,0.706,1.10,78.4,0.70,1.01,0.69,0.681,1.14,79.8,0.70,1.10,0.66,0.718,1.10,78.4,0.71,0.94,0.69,0.665,0.88,68.0,0.62,1,2,False,True,50000,0
4,Best of 3 (Online),2021-05-28,1622235600000,"$1,650",6+,South America (Online),Nuke,Dust2,Overpass,None,None,1.02,0.70,0.703,1.19,78.9,0.72,1.10,0.67,0.702,1.13,79.7,0.71,1.12,0.64,0.722,1.09,77.7,0.72,1.13,0.67,0.710,1.17,76.9,0.75,1.18,0.69,0.720,1.24,85.5,0.78,0.93,0.69,0.684,0.88,66.5,0.59,1.08,0.66,0.708,1.05,77.1,0.69,1.16,0.58,0.737,1.10,74.9,0.74,1.04,0.69,0.683,1.10,74.2,0.67,1.08,0.71,0.717,1.08,81.4,0.71,2,0,True,False,1650,0


In [12]:
df = df.drop(columns=["prize"])

In [13]:
df = df.drop(columns=["date"])

In [14]:
df.describe(include = 'object')

,best_of,teams,location,map1,map2,map3,map4,map5
count,5049,5044,5044,5049,4443,4429,98,98
unique,13,25,51,10,11,12,8,8
top,Best of 3,16,Europe (Online),Inferno,Inferno,Inferno,Mirage,Inferno
freq,2627,1438,3129,909,787,840,21,17


In [15]:
df.describe()

,timestamp,team1_player1_rating,team1_player1_DPR,team1_player1_KAST,team1_player1_impact,team1_player1_ADR,team1_player1_KPR,team1_player2_rating,team1_player2_DPR,team1_player2_KAST,team1_player2_impact,team1_player2_ADR,team1_player2_KPR,team1_player3_rating,team1_player3_DPR,team1_player3_KAST,team1_player3_impact,team1_player3_ADR,team1_player3_KPR,team1_player4_rating,team1_player4_DPR,team1_player4_KAST,team1_player4_impact,team1_player4_ADR,team1_player4_KPR,team1_player5_rating,team1_player5_DPR,team1_player5_KAST,team1_player5_impact,team1_player5_ADR,team1_player5_KPR,team2_player1_rating,team2_player1_DPR,team2_player1_KAST,team2_player1_impact,team2_player1_ADR,team2_player1_KPR,team2_player2_rating,team2_player2_DPR,team2_player2_KAST,team2_player2_impact,team2_player2_ADR,team2_player2_KPR,team2_player3_rating,team2_player3_DPR,team2_player3_KAST,team2_player3_impact,team2_player3_ADR,team2_player3_KPR,team2_player4_rating,team2_player4_DPR,team2_player4_KAST,team2_player4_impact,team2_player4_ADR,team2_player4_KPR,team2_player5_rating,team2_player5_DPR,team2_player5_KAST,team2_player5_impact,team2_player5_ADR,team2_player5_KPR,team1_points,team2_points,prize_money,prize_qualification
count,5.049000e+03,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5049.000000,5.049000e+03,5049.000000
mean,1.603618e+12,1.024054,0.670093,0.698236,1.048194,74.905962,0.684268,1.031270,0.673660,0.699205,1.050654,74.988691,0.685979,1.053662,0.668192,0.701344,1.071038,75.595762,0.694852,1.057267,0.670707,0.701820,1.068023,75.687800,0.694714,1.044845,0.673688,0.698482,1.047560,74.631293,0.684122,1.015262,0.674726,0.694750,1.036784,74.418023,0.678726,1.023971,0.677336,0.696135,1.040840,74.581303,0.681660,1.043506,0.671602,0.698293,1.055958,74.974510,0.688287,1.046710,0.676106,0.698277,1.055855,75.180234,0.688693,1.035005,0.678431,0.694688,1.037316,74.191563,0.679309,2.709249,2.425233,7.362391e+04,0.204793
std,1.165030e+10,0.075912,0.034409,0.021817,0.113044,4.625037,0.050615,0.074974,0.032619,0.022268,0.107789,4.637485,0.050689,0.080533,0.037521,0.022840,0.110301,4.802833,0.051555,0.080572,0.038519,0.023853,0.109502,4.506505,0.050372,0.086649,0.037614,0.026097,0.120509,5.341956,0.056756,0.076733,0.036002,0.023434,0.114848,4.655729,0.051329,0.077319,0.034602,0.023904,0.108852,4.654622,0.051025,0.084797,0.039170,0.025248,0.115719,4.944161,0.053553,0.084486,0.038338,0.025354,0.116143,4.840086,0.054385,0.093997,0.039877,0.029378,0.128508,5.680865,0.060995,4.207598,4.113052,1.708795e+05,0.403590
min,1.584244e+12,0.380000,0.580000,0.424000,0.170000,29.300000,0.260000,0.420000,0.540000,0.424000,0.390000,36.100000,0.320000,0.220000,0.540000,0.364000,0.160000,26.700000,0.180000,0.220000,0.540000,0.242000,0.230000,35.500000,0.240000,0.430000,0.530000,0.413000,0.280000,35.100000,0.280000,0.410000,0.580000,0.389000,0.310000,43.900000,0.360000,0.280000,0.560000,0.405000,0.200000,35.500000,0.240000,0.360000,0.480000,0.381000,0.120000,19.000000,0.140000,0.440000,0.540000,0.429000,0.150000,23.300000,0.200000,0.240000,0.580000,0.316000,0.110000,36.900000,0.250000,0.000000,0.000000,0.000000e+00,0.000000
25%,1.592901e+12,0.980000,0.650000,0.689000,0.980000,72.600000,0.660000,0.990000,0.650000,0.687000,0.980000,72.200000,0.660000,1.010000,0.650000,0.689000,1.020000,73.000000,0.670000

In [16]:
# now split to train and test
# split to 3/4 train and 1/4 test
train_rows = int(len(df) * 3 / 4)
test_rows = int(len(df) - train_rows)
train = df.sort_values("timestamp", ascending=True)[:train_rows]
test = df.sort_values("timestamp", ascending=True)[train_rows:]

train = train.drop(columns=["timestamp"])
test = test.drop(columns=["timestamp"])

In [17]:
train.head()

,best_of,teams,location,map1,map2,map3,map4,map5,team1_player1_rating,team1_player1_DPR,team1_player1_KAST,team1_player1_impact,team1_player1_ADR,team1_player1_KPR,team1_player2_rating,team1_player2_DPR,team1_player2_KAST,team1_player2_impact,team1_player2_ADR,team1_player2_KPR,team1_player3_rating,team1_player3_DPR,team1_player3_KAST,team1_player3_impact,team1_player3_ADR,team1_player3_KPR,team1_player4_rating,team1_player4_DPR,team1_player4_KAST,team1_player4_impact,team1_player4_ADR,team1_player4_KPR,team1_player5_rating,team1_player5_DPR,team1_player5_KAST,team1_player5_impact,team1_player5_ADR,team1_player5_KPR,team2_player1_rating,team2_player1_DPR,team2_player1_KAST,team2_player1_impact,team2_player1_ADR,team2_player1_KPR,team2_player2_rating,team2_player2_DPR,team2_player2_KAST,team2_player2_impact,team2_player2_ADR,team2_player2_KPR,team2_player3_rating,team2_player3_DPR,team2_player3_KAST,team2_player3_impact,team2_player3_ADR,team2_player3_KPR,team2_player4_rating,team2_player4_DPR,team2_player4_KAST,team2_player4_impact,team2_player4_ADR,team2_player4_KPR,team2_player5_rating,team2_player5_DPR,team2_player5_KAST,team2_player5_impact,team2_player5_ADR,team2_player5_KPR,team1_points,team2_points,team1_wins,team2_wins,prize_money,prize_qualification
4967,Best of 3 (Online),3+,Oceania (Online),Default,Inferno,Vertigo,None,None,0.94,0.75,0.665,0.98,71.2,0.63,0.93,0.76,0.657,0.98,71.5,0.63,1.09,0.68,0.691,1.09,80.6,0.72,1.13,0.64,0.709,1.16,77.0,0.73,0.95,0.71,0.670,0.96,69.6,0.63,0.99,0.69,0.691,0.93,72.5,0.64,1.02,0.68,0.684,1.04,75.0,0.70,1.02,0.71,0.680,1.07,74.1,0.67,0.80,0.74,0.640,0.93,71.0,0.56,1.06,0.70,0.702,1.05,78.2,0.71,1,2,False,True,0,1
4969,Best of 1,18,Europe (Online),Default,None,None,None,None,0.94,0.69,0.683,0.83,70.1,0.58,1.10,0.65,0.703,1.22,77.4,0.75,1.03,0.67,0.702,0.98,72.8,0.67,0.99,0.65,0.708,0.89,68.9,0.61,0.99,0.68,0.694,0.95,69.5,0.64,1.14,0.70,0.688,1.29,82.8,0.75,1.06,0.66,0.713,1.01,76.1,0.67,0.91,0.72,0.653,0.95,70.8,0.64,1.07,0.66,0.690,1.15,75.0,0.73,1.06,0.67,0.695,1.03,75.9,0.70,1,0,True,False,0,1
4968,Best of 3 (Online),2+,Europe (Online),Nuke,Train,Overpass,None,None,1.05,0.64,0.705,1.01,71.7,0.67,0.99,0.71,0.687,0.93,73.7,0.67,0.91,0.71,0.667,0.83,70.4,0.62,0.93,0.71,0.667,0.87,69.3,0.60,0.98,0.69,0.694,0.94,70.3,0.63,0.94,0.70,0.657,0.98,75.3,0.66,1.02,0.72,0.707,1.03,74.1,0.67,0.92,0.70,0.684,0.97,71.2,0.63,0.98,0.70,0.671,1.00,69.3,0.65,0.93,0.70,0.677,0.91,67.8,0.56,0,2,False,True,1500,0
4970,Best of 3,8,Europe (Online),Dust2,Train,Mirage,None,None,0.90,0.66,0.695,0.87,66.4,0.59,0.99,0.69,0.707,1.07,80.2,0.69,1.07,0.65,0.701,1.16,73.9,0.73,1.12,0.68,0.728,1.12,81.4,0.72,1.06,0.66,0.706,1.03,76.0,0.69,1.00,0.71,0.672,1.04,73.3,0.66,1.11,0.63,0.706,1.12,74.4,0.72,1.01,0.67,0.683,1.00,72.1,0.67,1.01,0.67,0.679,0.96,73.3,0.67,0.93,0.67,0.686,0.84,66.2,0.59,1,2,False,True,10000,0
4971,Best of 3,12,North America (Online),Train,Nuke,Inferno,None,None,0.89,0.68,0.689,0.85,68.0,0.61,1.10,0.70,0.693,1.19,80.7,0.72,1.13,0.63,0.709,1.19,74.2,0.73,1.05,0.69,0.699,1.16,77.6,0.72,1.16,0.65,0.728,1.17,81.4,0.75,1.09,0.64,0.689,1.03,76.6,0.74,1.11,0.65,0.706,1.08,76.7,0.76,0.97,0.68,0.686,0.94,71.7,0.66,0.98,0.65,0.696,0.88,70.4,0.67,1.12,0.62,0.698,1.10,72.0,0.74,1,2,False,True,1000000,0


In [18]:
test.head()

,best_of,teams,location,map1,map2,map3,map4,map5,team1_player1_rating,team1_player1_DPR,team1_player1_KAST,team1_player1_impact,team1_player1_ADR,team1_player1_KPR,team1_player2_rating,team1_player2_DPR,team1_player2_KAST,team1_player2_impact,team1_player2_ADR,team1_player2_KPR,team1_player3_rating,team1_player3_DPR,team1_player3_KAST,team1_player3_impact,team1_player3_ADR,team1_player3_KPR,team1_player4_rating,team1_player4_DPR,team1_player4_KAST,team1_player4_impact,team1_player4_ADR,team1_player4_KPR,team1_player5_rating,team1_player5_DPR,team1_player5_KAST,team1_player5_impact,team1_player5_ADR,team1_player5_KPR,team2_player1_rating,team2_player1_DPR,team2_player1_KAST,team2_player1_impact,team2_player1_ADR,team2_player1_KPR,team2_player2_rating,team2_player2_DPR,team2_player2_KAST,team2_player2_impact,team2_player2_ADR,team2_player2_KPR,team2_player3_rating,team2_player3_DPR,team2_player3_KAST,team2_player3_impact,team2_player3_ADR,team2_player3_KPR,team2_player4_rating,team2_player4_DPR,team2_player4_KAST,team2_player4_impact,team2_player4_ADR,team2_player4_KPR,team2_player5_rating,team2_player5_DPR,team2_player5_KAST,team2_player5_impact,team2_player5_ADR,team2_player5_KPR,team1_points,team2_points,team1_wins,team2_wins,prize_money,prize_qualification
1276,Best of 3 (Online),4,North America (Online),Train,Inferno,Nuke,None,None,1.08,0.61,0.716,1.12,73.6,0.71,1.09,0.71,0.707,1.19,80.2,0.70,1.07,0.62,0.725,0.95,72.3,0.67,1.06,0.72,0.702,1.09,76.9,0.71,1.08,0.69,0.709,1.09,77.8,0.70,1.09,0.64,0.722,1.15,78.7,0.74,1.12,0.65,0.726,1.11,79.5,0.71,1.01,0.67,0.703,0.98,71.7,0.64,1.13,0.63,0.726,1.11,75.8,0.73,1.08,0.69,0.710,1.09,77.1,0.71,0,2,False,True,0,1
1277,Best of 3 (Online),4,North America (Online),Dust2,Inferno,Nuke,None,None,1.08,0.68,0.699,1.15,75.0,0.71,1.09,0.68,0.711,1.11,76.1,0.73,0.96,0.69,0.693,0.90,69.0,0.61,1.12,0.64,0.714,1.15,76.0,0.72,1.05,0.69,0.705,1.03,76.3,0.68,1.00,0.64,0.707,0.90,68.7,0.61,1.12,0.63,0.714,1.13,73.9,0.74,1.14,0.65,0.734,1.08,81.6,0.75,1.08,0.67,0.720,1.02,77.0,0.70,1.05,0.67,0.722,1.04,72.3,0.67,2,1,True,False,0,1
1278,Best of 3 (Online),32,Europe (Online),Vertigo,Inferno,Nuke,None,None,1.13,0.64,0.703,1.23,78.5,0.77,0.99,0.67,0.708,1.04,74.4,0.67,1.04,0.67,0.704,1.04,72.9,0.66,1.04,0.65,0.707,0.96,72.0,0.67,1.20,0.67,0.722,1.29,87.5,0.77,0.95,0.68,0.703,0.98,72.2,0.65,1.01,0.65,0.716,0.99,75.8,0.68,1.08,0.62,0.705,1.06,70.9,0.70,1.00,0.69,0.696,0.99,71.9,0.63,0.99,0.68,0.694,0.95,69.5,0.64,0,2,False,True,100000,0
1279,Best of 3 (Online),4+,Poland (Online),Nuke,Mirage,Overpass,None,None,0.92,0.71,0.667,0.92,66.2,0.61,0.97,0.68,0.675,0.89,71.8,0.62,0.93,0.72,0.643,0.95,67.9,0.64,0.90,0.74,0.635,0.92,68.0,0.62,0.80,0.76,0.631,0.75,63.0,0.56,0.97,0.72,0.678,0.93,75.3,0.63,1.06,0.66,0.678,1.13,71.7,0.69,0.93,0.72,0.662,0.90,71.2,0.62,0.84,0.73,0.657,0.76,61.2,0.55,1.02,0.66,0.705,1.02,70.7,0.65,1,2,False,True,5192,0
1280,Best of 3 (Online),4+,Ukraine (Online),Nuke,Dust2,Train,None,None,1.07,0.69,0.670,1.24,75.6,0.75,0.97,0.77,0.616,1.17,76.3,0.70,0.96,0.61,0.699,0.80,64.0,0.58,1.07,0.73,0.688,1.17,80.5,0.73,0.96,0.70,0.698,0.91,68.6,0.61,1.08,0.61,0.704,1.03,71.3,0.68,1.05,0.68,0.691,1.07,75.4,0.66,1.04,0.68,0.676,1.08,74.1,0.70,0.98,0.64,0.703,0.89,66.4,0.60,1.02,0.65,0.706,0.94,71.5,0.65,1,2,False,True,11790,0


In [19]:
train.to_csv("train.csv",index=False)
test.to_csv("test.csv",index=False)